# 3D Reconstruction with ASTRA Toolkit

This notebook demonstrates how to use the `AstraReconstructor` class to perform 3D reconstruction from 2D TIFF images using the ASTRA toolkit. The class supports both SIRT and FBP reconstruction algorithms and provides various visualization capabilities.

## 1. Import Required Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astra_reconstruction import AstraReconstructor

# For displaying plots inline
%matplotlib inline

## 2. Initialize the Reconstructor

First, we'll initialize the `AstraReconstructor` class with the provided configuration files.

In [ ]:
# Paths to configuration files
config_json = 'CT4000.json'
config_ini = 'ScanAllParam.ini'

# Initialize the reconstructor
reconstructor = AstraReconstructor(config_json=config_json, config_ini=config_ini)

# Set volume size for reconstruction (adjust as needed)
reconstructor.set_volume_size([256, 256, 256])

# Set algorithm parameters
reconstructor.set_algorithm_params(algorithm='SIRT', iterations=100)

## 3. Load Projection Data

Next, we'll load the projection data from TIFF files. Replace `'path/to/tiff/files'` with the actual path to your TIFF files.

In [ ]:
# Path to TIFF files
tiff_dir = 'path/to/tiff/files'

# Load projections
try:
    projections = reconstructor.load_projections(tiff_dir, pattern='*.tif', normalize=True)
    print(f"Projections loaded with shape: {projections.shape}")
except ValueError as e:
    print(f"Error: {e}")
    print("Please update the path to your TIFF files.")

## 4. Perform 3D Reconstruction

Now we'll perform the 3D reconstruction using the SIRT algorithm. You can also use FBP by changing the algorithm parameter.

In [ ]:
# For demonstration purposes, if no projections are available, create synthetic data
if reconstructor.projections is None:
    print("Creating synthetic projection data for demonstration...")
    # Create a simple phantom
    vol_shape = reconstructor.config['volume_size']
    phantom = np.zeros(vol_shape)
    
    # Add some spheres to the phantom
    center = np.array(vol_shape) // 2
    radius = min(vol_shape) // 4
    
    x, y, z = np.ogrid[:vol_shape[0], :vol_shape[1], :vol_shape[2]]
    dist = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)
    phantom[dist <= radius] = 1.0
    
    # Create a smaller sphere inside
    small_radius = radius // 2
    small_center = center + np.array([radius//2, 0, 0])
    dist = np.sqrt((x - small_center[0])**2 + (y - small_center[1])**2 + (z - small_center[2])**2)
    phantom[dist <= small_radius] = 2.0
    
    # Create projections
    import astra
    vol_id = astra.data3d.create('-vol', reconstructor.vol_geom, data=phantom)
    proj_id = astra.data3d.create('-sino', reconstructor.proj_geom)
    
    # Create forward projection
    cfg = astra.astra_dict('FP3D_CUDA')
    cfg['VolumeDataId'] = vol_id
    cfg['ProjectionDataId'] = proj_id
    fp_id = astra.algorithm.create(cfg)
    astra.algorithm.run(fp_id)
    
    # Get the projections
    reconstructor.projections = astra.data3d.get(proj_id)
    
    # Add some noise
    np.random.seed(42)
    noise = np.random.normal(0, 0.05, reconstructor.projections.shape)
    reconstructor.projections += noise
    reconstructor.projections = np.clip(reconstructor.projections, 0, None)
    
    # Clean up
    astra.algorithm.delete(fp_id)
    astra.data3d.delete(vol_id)
    astra.data3d.delete(proj_id)
    
    print(f"Created synthetic projections with shape: {reconstructor.projections.shape}")

In [ ]:
# Perform reconstruction
try:
    # SIRT reconstruction
    volume_sirt = reconstructor.reconstruct(algorithm='SIRT', iterations=100, gpu_index=0)
    print(f"SIRT reconstruction completed with volume shape: {volume_sirt.shape}")
    
    # Save the reconstructed volume (optional)
    # reconstructor.save_volume('sirt_reconstruction.npy')
except Exception as e:
    print(f"Error during reconstruction: {e}")

In [ ]:
# FBP reconstruction
try:
    volume_fbp = reconstructor.reconstruct(algorithm='FBP', gpu_index=0)
    print(f"FBP reconstruction completed with volume shape: {volume_fbp.shape}")
    
    # Save the reconstructed volume (optional)
    # reconstructor.save_volume('fbp_reconstruction.npy')
except Exception as e:
    print(f"Error during reconstruction: {e}")

## 5. Visualize the Results

Now we'll visualize the reconstructed volume using various visualization methods.

In [ ]:
# Visualize a slice of the volume
try:
    fig = reconstructor.visualize_slice(axis=2, slice_index=None, figsize=(10, 8), cmap='gray')
    plt.show()
except Exception as e:
    print(f"Error during slice visualization: {e}")

In [ ]:
# Visualize orthogonal views
try:
    fig = reconstructor.create_orthogonal_views(slice_indices=None, figsize=(15, 5), cmap='gray')
    plt.show()
except Exception as e:
    print(f"Error during orthogonal views visualization: {e}")

In [ ]:
# Visualize Maximum Intensity Projection (MIP)
try:
    fig = reconstructor.visualize_mip(axis=2, figsize=(10, 8), cmap='gray')
    plt.show()
except Exception as e:
    print(f"Error during MIP visualization: {e}")

In [ ]:
# Install scikit-image if not already installed (needed for 3D visualization)
try:
    import skimage
except ImportError:
    !pip install scikit-image

In [ ]:
# 3D Visualization (isosurface)
try:
    fig = reconstructor.visualize_volume_3d(threshold=0.5, figsize=(12, 10))
    plt.show()
except Exception as e:
    print(f"Error during 3D visualization: {e}")

## 6. Compare SIRT and FBP Reconstructions

Let's compare the results of SIRT and FBP reconstructions.

In [ ]:
# Compare SIRT and FBP reconstructions
try:
    # Save the SIRT volume
    sirt_volume = reconstructor.volume.copy()
    
    # Perform FBP reconstruction if not done already
    if 'volume_fbp' not in locals():
        volume_fbp = reconstructor.reconstruct(algorithm='FBP', gpu_index=0)
    
    # Get the FBP volume
    fbp_volume = reconstructor.volume.copy()
    
    # Create a figure to compare slices
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    # Get the middle slice
    slice_index = sirt_volume.shape[2] // 2
    
    # Display SIRT slice
    im0 = axes[0].imshow(sirt_volume[:, :, slice_index], cmap='gray')
    axes[0].set_title(f'SIRT Reconstruction (Z={slice_index})')
    fig.colorbar(im0, ax=axes[0])
    
    # Display FBP slice
    im1 = axes[1].imshow(fbp_volume[:, :, slice_index], cmap='gray')
    axes[1].set_title(f'FBP Reconstruction (Z={slice_index})')
    fig.colorbar(im1, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
    # Restore the SIRT volume for further visualization
    reconstructor.volume = sirt_volume
except Exception as e:
    print(f"Error during comparison: {e}")

## 7. Advanced Usage: Custom Visualization

You can also create custom visualizations using the reconstructed volume.

In [ ]:
# Custom visualization: Multi-slice view
try:
    if reconstructor.volume is not None:
        # Create a figure with multiple slices
        num_slices = 4
        fig, axes = plt.subplots(1, num_slices, figsize=(15, 4))
        
        # Get slice indices
        z_size = reconstructor.volume.shape[2]
        slice_indices = np.linspace(z_size // 5, 4 * z_size // 5, num_slices, dtype=int)
        
        # Display slices
        for i, slice_idx in enumerate(slice_indices):
            im = axes[i].imshow(reconstructor.volume[:, :, slice_idx], cmap='gray')
            axes[i].set_title(f'Slice Z={slice_idx}')
            fig.colorbar(im, ax=axes[i])
        
        plt.tight_layout()
        plt.show()
except Exception as e:
    print(f"Error during custom visualization: {e}")

## 8. Conclusion

This notebook demonstrated how to use the `AstraReconstructor` class to perform 3D reconstruction from 2D TIFF images using the ASTRA toolkit. The class provides a convenient interface for loading projection data, performing reconstruction using SIRT and FBP algorithms, and visualizing the results in various ways.

To use this with your own data:

1. Update the paths to your configuration files and TIFF images
2. Adjust the volume size and algorithm parameters as needed
3. Run the reconstruction and visualization cells

For more advanced usage, you can modify the `AstraReconstructor` class to add additional functionality or customize the existing methods.